In [26]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

class SklModel():
    # dataframe = None

    def __init__(self):
        self.model=None
        self.columns = None
        self.test_parameters = {}
        self.model_parameters = {}
        self.dataframe= None
        # self.dataframe_org= None
        self.filepath = None

    #step1 csvファイルの読込
    def set_filepath(self, filepath):
        self.filepath = filepath
        self.dataframe = pd.read_csv(filepath) # CSV読込
        # self.dataframe_org=self.dataframe.copy()
        return self.dataframe

    #step2 ダミー変数の設定
    def get_dummy_columns(self, df:pd.DataFrame, columns:list, drop=0):
        drop_flag = {0:False,1:True}
        # print(drop_flag[drop])
        for column in columns:
            dummy = pd.get_dummies(
                df[column], drop_first=drop_flag[drop], dtype=int)
            df = pd.concat([df, dummy], axis=1)
        df = df.drop(columns, axis=1)
        return df

    def set_test_parameters(self, parameters):
        self.test_parameters = parameters

    #step3
    def split(self,*parrays):
        # データを分割する
        return train_test_split(*parrays, **self.test_parameters)

    def set_target(self, target_column):
        self.target_column = target_column

        # 特徴量と正解ラベルを分割する
        if self.columns is None:
            columns = self.dataframe.drop(
                self.target_column, axis=1).columns.tolist()
            self.set_columns(columns)
        
        return self.target_column

    def get_target(self):
        return self.target_column

    def set_columns(self, columns):
        self.columns = columns
        return self.columns
        # return cls.columns := columns

    def get_columns(self):
        return self.columns
    
    def fillna_meaning(self, df) -> pd.DataFrame:
        return df.fillna(df.mean())
    
    def ss_transform(self, df, model=None):
        if (model is None):
            sc_model_x = StandardScaler() #訓練データxの標準化モデル
            sc_model_x.fit(df)
        else:
            sc_model_x = model
        
        # 各列のデータを標準化してsc_xに代入
        sc_x = sc_model_x.transform(df) #標準化されたxのdfデータ
        sc_x # 表示

        return sc_x, sc_model_x

    def set_model_parameters(self, parameters):
        self.model_parameters = parameters

    def get_model_parameters(self, parameters):
        return self.model_parameters

    def init_model(self,model_name=""):
        if(model_name=="DecisionTreeClassifier"):
            return DecisionTreeClassifier(**self.model_parameters)
        elif(model_name=="LinearRegression"):
            return LinearRegression()
        else:
            return None



#8-1 CSVの読込
import matplotlib.pyplot as plt #plot用の初期化
%matplotlib inline
sm=SklModel()
df = sm.set_filepath("../datafiles/Boston.csv")
# print(df.head(2))

#8-3 CRIMEの調査
df['CRIME'].value_counts()

#8-4 ダミー変数設定(自動化候補)
df2 = sm.get_dummy_columns(df, ['CRIME'], 1)
df2

#8-5 columnsもtargetも設定が無いので全てのcolumnsを分割
sm.set_test_parameters({"test_size": 0.2, "random_state": 0})
train_val,test = sm.split(df2)
train_val,test

#8-6 欠損値があるかどうかの確認
train_val.isnull().sum()

#8-7 欠損値を平均値で穴埋め（自動化候補）
train_val2 = sm.fillna_meaning(train_val)
train_val2

# #8-8-2 外れ値の確認
# fig = plt.figure(figsize=(8,10))
# colname = train_val2.columns
# colname
# # for name in colname[:13]:
# for n, col in enumerate(colname):
#     train_val2.plot(
#         ax=fig.add_subplot(5,3,n+1), kind='scatter', x= col, y='PRICE', s=3)
# plt.tight_layout()
# plt.show()

#8-9 外れ値が存在するインデックスの確認
# RMの外れ値
out_line1 = train_val2[(train_val2['RM'] < 6) &
(train_val2['PRICE'] > 40)].index
# PTRATIOの外れ値
out_line2 = train_val2[(train_val2['PTRATIO'] > 18) &
    (train_val2['PRICE'] > 40)].index
# print(out_line1, out_line2)

#8-10 外れ値の削除
train_val3 = train_val2.drop([76], axis = 0)

#8-11 予測に使用する特徴量の列以外を取り除く
col = ['INDUS', 'NOX', 'RM', 'PTRATIO', 'LSTAT', 'PRICE']
train_val4 = train_val3[col]
train_val4.head(3)

#8-12 各列同士の相関係数を調べる
train_val4.corr()

#8-13 PRICE列との相関係数を調べる
train_cor = train_val4.corr()['PRICE']
train_cor

#8-16 各要素を絶対値に変換
abs_cor = train_cor.map(abs)
abs_cor

#8-17 降順
abs_cor.sort_values(ascending = False)

#8-18 データ分割して訓練データと検証データに
col =sm.set_columns(['RM', 'LSTAT', 'PTRATIO'])
target = sm.set_target(['PRICE'])
x = train_val4[col]
t = train_val4[target]
x_train, x_val, y_train, y_val = sm.split(x,t)
x_train, x_val, y_train, y_val

#8-19 データ標準化
# sc_model_x = StandardScaler() #訓練データxの標準化モデル
# sc_model_x.fit(x_train)

# # 各列のデータを標準化してsc_xに代入
# sc_x = sc_model_x.transform(x_train) #標準化されたxのdfデータ
# sc_x # 表示
sc_x, sc_model_x = sm.ss_transform(x_train)
sc_x, sc_model_x

#8-20 見やすくして平均値0（ほぼ0）を確認 = 標準化された
# array 型だと見づらいのでデータフレームに変換
tmp_df = pd.DataFrame(sc_x, columns = x_train.columns)
# 平均値の計算
tmp_df.mean()

#8-21 標準偏差の計算
tmp_df.std() # 標準偏差の計算

#8-22 正解データを標準化
sc_y,sc_model_y = sm.ss_transform(y_train)

#8-23 標準化したデータで学習
sm.set_model_parameters([])
model=sm.init_model("LinearRegression")
model.fit(sc_x,sc_y)

#8-24 決定係数を求める
model.score(x_val,y_val)



c:\Users\user\anaconda3\envs\py39\lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


-13.085044375040093

In [ ]:
# #8-41モデルの保存
# import pickle
# with open('boston.pkl',"wb") as f:
#     pickle.dump(model,f) #モデルの保存
# with open('boston_scx.pkl','wb') as f:
#     pickle.dump(sc_model_x2,f) #xの標準化モデルの保存
# with open('boston_scy.pkl','wb') as f:
#     pickle.dump(sc_model_y2,f) #yの標準化モデルの保存
